In [5]:
import os
from dotenv import load_dotenv

from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.tools.retriever import create_retriever_tool

from langchain_google_community import GoogleSearchAPIWrapper
from langchain.tools import Tool

from langchain import hub
from langchain.agents import create_react_agent, AgentExecutor

'''API KEY LOADING'''
# Load api key
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
GOOGLE_CSE_ID = os.getenv("GOOGLE_CSE_ID")


'''RAG TOOL'''
# 1. Load Data (txt 파일 문서)
loader = TextLoader(file_path='졸업이수규정.txt', encoding='utf-8')
text = loader.load()
text_content = text[0].page_content

# 2. Text Split (Txt doc -> Small chunks(texts); RecursiveCharacterTextSplitter 활용)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 150,
    chunk_overlap = 50,
    length_function = len
)

texts = text_splitter.split_text(text_content)

# 3. Indexing (Texts -> Embedding -> Vector Store; Chroma, OpenAIEmbeddings, 유사도 기반 검색(similarity search) 활용)
vectorstore = Chroma.from_texts(
    texts=texts, 
    embedding=OpenAIEmbeddings(api_key=OPENAI_API_KEY)
    )

retriever = vectorstore.as_retriever()

# 4. RAG Tool Generation
retriever_tool = create_retriever_tool(
    retriever=retriever,
    name="graduation_rules_retriever",
    description="서울대학교 산업공학과의 졸업을 위한 이수 규정, 필요 학점, 전공 필수, 교양 학점 등 졸업 요건에 대한 질문에 답할 때 사용"
)


'''WEB SEARCH TOOL'''
search = GoogleSearchAPIWrapper(google_api_key=GOOGLE_API_KEY, google_cse_id=GOOGLE_CSE_ID)

search_tool = Tool(
    func=search.run,
    name='google_web_search',
    description='최신, 실시간 정보, 일반적인 정보 등 외부 웹 검색이 필요할 때 사용'
)


'''REACT AGENT'''
tools = [retriever_tool, search_tool]

llm = ChatOpenAI(
    model='gpt-4-turbo',
    api_key=OPENAI_API_KEY,
    temperature=0

)

prompt = hub.pull("hwchase17/react")

agent = create_react_agent(llm, tools, prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [7]:
'''TEST'''
response_rag = agent_executor.invoke({
    "input" : "서울대학교 공과대학 산업공학과를 졸업하려면 교양을 몇 학점 이수해야 해?"
})
print(f"답변 : {response_rag['output']}")



> Entering new AgentExecutor chain...
To find the specific graduation requirements for the Department of Industrial Engineering at Seoul National University, especially the required credits for liberal arts courses, I should use the specialized tool designed to retrieve such academic information.

Action: graduation_rules_retriever
Action Input: "서울대학교 산업공학과 교양 학점 이수 요건"서울대학교 공과대학 산업공학과 졸업이수규정

- 졸 업 학 점 : 130학점
- 교양이수학점 : 50학점
- 전 공 : 59학점
- 전 공 필 수 : 28학점
- 학과개설 전공선택 : 21학점 이상

서울대학교 공과대학 산업공학과 졸업이수규정

- 졸 업 학 점 : 130학점
- 교양이수학점 : 50학점
- 전 공 : 59학점
- 전 공 필 수 : 28학점
- 학과개설 전공선택 : 21학점 이상

교과목 이수규정
대학원세미나수업 1학점을 이수하여야 함(2004학번부터, 2006년 3월 개정)
- 세미나과목은 1학기에만 개설됨
- 대학원 수업은 3학년 이상만 수강가능

교과목 이수규정
대학원세미나수업 1학점을 이수하여야 함(2004학번부터, 2006년 3월 개정)
- 세미나과목은 1학기에만 개설됨
- 대학원 수업은 3학년 이상만 수강가능I now know the final answer

Final Answer: 서울대학교 공과대학 산업공학과를 졸업하기 위해서는 교양 과목으로 총 50학점을 이수해야 합니다.

> Finished chain.
답변 : 서울대학교 공과대학 산업공학과를 졸업하기 위해서는 교양 과목으로 총 50학점을 이수해야 합니다.


In [8]:
'''TEST'''
response_search = agent_executor.invoke({
    "input" : "서울대학교 공과대학 산업공학과를 졸업하기 위한 이수 규정을 알고 싶은데, 해당 정보를 확인할 수 있는 웹페이지를 알려줘."
})
print(f"답변 : {response_search['output']}")



> Entering new AgentExecutor chain...
To find the specific webpage containing the graduation requirements for the Department of Industrial Engineering at Seoul National University's College of Engineering, I should use a web search tool.

Action: google_web_search
Action Input: "서울대학교 공과대학 산업공학과 졸업 이수 규정 웹페이지"서울대학교 공과대학입니다. 서울대 공대소식을 확인해보세요. 대학 간 복수전공인 경우에는 소속 학과(부)장의 승인을 받아 학장을 경유하여 복수전공을 이수하고자 하는 대학에 제출합니다. ... 공과대학 (산업공학과). 공과대학, ... Jul 4, 2025 ... ... 서울대학교 수강신청 사이트. (https://sugang.snu.ac.kr/)에서 ... 다만'서울대학교유급규정'을 적용하는 대학은 예외로 한다. 라. 의과대학 ... 공지사항 · 학부대학 · 자유전공학부 · 인문대학 · 국어국문학과 · 중어중문학과 · 영어영문학과 · 불어불문학과 · 독어독문학과. 대학/대학원. 대학/대학원학생 성장의요람, 새로운 가치를 만드는 대학; 대학/학부 열림 닫힘 · 인문대학 · 사회과학대학 · 경상대학 · 사범대학 · 생명자원과학대학 ... Jul 29, 2025 ... ... 대학 교류학생 지원 안내 ￭ 서울대학교 캠퍼스(관악) 안내. Ⅲ. 공통교육 ... 이수 관련 안내 ... Today(3 개) · 17,507.9천원. 학생 1인당 교육비 (2024년 기준, 대학정보공시) · 0.0%. 신입생 충원율(정원내) · 0%. 재학생 충원율(정원내) · 0.0%. 취업률 · 0.0천원. 재학생 ... Jul 3, 2024 ... 수강정보는 교양 전공 대학원과정으로 구분되어 있으며 서울대학교 수강신청 사이. ,. ,. ,.